<a href="https://colab.research.google.com/github/thadigitalguru/PDFtoJSON-OpenAI/blob/main/PDFtoJSONTool.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**PIP install required Libraries.**

For this demo we are using:

*   openai - to call the LLM
*   pdfminer.six - to extract text from a PDF.


In [ ]:
!pip install openai pdfminer.six

**Import the OpenAI API Key from Colab Secrets**

We need to get the OpenAI from the secrets in Google Colab and put it in 'api_key' and then create a 'client' object that will be used later.


In [ ]:
from google.colab import userdata
import openai

api_key = userdata.get("OpenAI")

if not api_key:
    raise ValueError("❌ OpenAI API key not found. Use `userdata.set('OpenAI', 'sk-...')` first.")

client = openai.OpenAI(api_key=api_key)


**Upload the PDF File to Process**

We use the files library from google colab to upload the file we want to process.


In [ ]:
from google.colab import files

uploaded = files.upload()  # Upload a health report PDF
pdf_filename = list(uploaded.keys())[0]
print(f"📄 Uploaded: {pdf_filename}")


**Define the Function Schema**

The function schema is like a contract or blueprint that tells GPT:

“You are not just generating text. You are calling a function named extract_patient_info, and here’s what the input should look like.”

It's based on JSON Schema standards and includes:

*   **"name"**	The name of the virtual function GPT is calling
*   **"description"**	Helps the model understand what the function is for
*   **"parameters"**	Specifies the expected structure of the output (a JSON object)
*   **"properties"**	Lists all fields we want from the report
*   **"required"**	Tells GPT which fields are mandatory


In [ ]:
patient_info_function = {
    "name": "extract_patient_info",
    "description": "Extracts key personal fields from a health report.",
    "parameters": {
        "type": "object",
        "properties": {
            "name": {"type": "string"},
            "patient_id": {"type": "string"},
            "mrn": {"type": ["string", "null"]},
            "birth_date": {"type": "string", "format": "date"},
            "age": {"type": "integer"},
            "gender": {"type": "string"},
            "address": {"type": "string"},
            "phone": {
                "type": "object",
                "properties": {
                    "home": {"type": ["string", "null"]},
                    "work": {"type": ["string", "null"]},
                    "cell": {"type": ["string", "null"]}
                }
            },
            "email": {"type": "string"},
            "provider": {"type": "string"},
            "referring_provider": {"type": "string"},
            "print_date": {"type": "string", "format": "date"}
        },
        "required": ["name", "patient_id", "birth_date", "age", "gender", "address", "email"]
    }
}

**Define PDF and OpenAI Functions**

We use the **extract_text** library function to get the text from the PDF that was uploads.


In [ ]:
from pdfminer.high_level import extract_text

def extract_text_from_pdf(path: str) -> str:
    return extract_text(path)

pdf_text = extract_text_from_pdf(pdf_filename)
print("✅ PDF text extracted. Sample preview:\n")
print(pdf_text[:1000])


**Extract Data**

We are using the gpt-4o-mini to extract data and put the results into function_schema.


In [ ]:
import json

def call_extraction(text: str, function_schema: dict) -> dict:
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        temperature=0.0,
        messages=[
            {"role": "system", "content": "You are a medical data extractor."},
            {"role": "user", "content": text}
        ],
        functions=[function_schema],
        function_call={"name": function_schema["name"]}
    )
    args = response.choices[0].message.function_call.arguments
    return json.loads(args)

try:
    patient_info = call_extraction(pdf_text, patient_info_function)
    print("✅ Patient Info Extracted:\n")
    print(json.dumps(patient_info, indent=2))
except Exception as e:
    print("❌ Extraction failed:", str(e))

